<a href="https://colab.research.google.com/github/aevfly/video_upscale/blob/main/GFPGAN_Video_upscale%20v1.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Important Notes

* If you are using the free Colab tier , it works best for short length videos
* Just follow the instructions in the notebook and you should be fine!

*Colab Notebook prepared by AEV*

In [1]:
%cd /content
!rm -rf GFPGAN
!git clone https://github.com/TencentARC/GFPGAN.git
%cd GFPGAN

# Set up the environment
# Install basicsr - https://github.com/xinntao/BasicSR
# We use BasicSR for both training and inference
!pip install basicsr
# Install facexlib - https://github.com/xinntao/facexlib
# We use face detection and face restoration helper in the facexlib package
!pip install facexlib
# Install other depencencies
!pip install -r requirements.txt
!python setup.py develop
!pip install realesrgan  # used for enhancing the background (non-face) regions

# Download the pre-trained model
#!wget https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth -P experiments/pretrained_models

# Now we use the V1.3 model for the demo
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models

# Download the pre-trained GFPGAN v1.4 model
#!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/GFPGANv1.4.pth -P experiments/pretrained_models


/content
Cloning into 'GFPGAN'...
remote: Enumerating objects: 527, done.
remote: Counting objects: 100% (212/212), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 527 (delta 170), reused 154 (delta 154), pack-reused 315 (from 2)
Receiving objects: 100% (527/527), 5.38 MiB | 20.85 MiB/s, done.
Resolving deltas: 100% (281/281), done.
/content/GFPGAN
/usr/local/lib/python3.11/dist-packages/setuptools/__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/usr/local/lib/python3.11/dist-packages/setuptools/command/develop.py:41: EasyInstallDeprecationWar

# 2. This section is for creating some important folders (IMPORTANT : Please Read)



*   First Run this cell

*   Once you've run the cell , you can see that a folder named "videos" will be created inside the GFPGAN folder


*   Just drag and drop any number of videos from your computer and place them inside the "videos" folder. (videos you want to upscale)







In [2]:
import os
from google.colab import files
import shutil

upload_folder = 'upload'
result_folder = 'results'
video_folder = 'videos'
video_result_folder = 'results_videos'
video_mp4_result_folder = 'results_mp4_videos'
directory = '/content/GFPGAN/videos'
preprocessed_dir = '/content/GFPGAN/videos_preprocessed'

if os.path.isdir(upload_folder):
  print(upload_folder+" exists")
else :
  os.mkdir(upload_folder)

if os.path.isdir(video_folder):
  print(video_folder+" exists")
else :
  os.mkdir(video_folder)

if os.path.isdir(video_result_folder):
  print(video_result_folder+" exists")
else :
  os.mkdir(video_result_folder)

if os.path.isdir(video_mp4_result_folder):
  print(video_mp4_result_folder+" exists")
else :
  os.mkdir(video_mp4_result_folder)

if os.path.isdir(result_folder):
  print(result_folder+" exists")

else :
  os.mkdir(result_folder)




# 3. Inference (Please READ)



*   Then run this cell , it will take a good amount of time depending on the length of your video and how many videos you want to upscale

*   Once the upscaling process is complete , you can find your upscaled video results , under the "results_mp4_videos" folder.



*   By default the video is upscaled to a 25fps format,  in case you want to change that , it can be changed within the cell below ,
change the "fps" variable. (Can be found  towards the end part of the code section.)








In [3]:
import cv2
import numpy as np
import glob
from os.path import isfile, join
import subprocess
from IPython.display import clear_output


# Patch basicsr degradations.py to fix torchvision import
file_path = "/usr/local/lib/python3.11/dist-packages/basicsr/data/degradations.py"
with open(file_path, 'r') as file:
    content = file.read()
content = content.replace(
    "from torchvision.transforms.functional_tensor import rgb_to_grayscale",
    "from torchvision.transforms.functional import rgb_to_grayscale"
)
with open(file_path, 'w') as file:
    file.write(content)
print("Patched degradations.py successfully.")

# Ensure latest GFPGAN and Real-ESRGAN packages
!pip install gfpgan --upgrade
!pip install realesrgan --upgrade

for filename in os.listdir(directory):
    if filename.endswith(('.mp4', '.avi', '.mov')):
        input_path = os.path.join(directory, filename)
        output_path = os.path.join(preprocessed_dir, f"pre_{filename}")
        !ffmpeg -i {input_path} -vf scale=960:540:flags=lanczos {output_path}

video_result_folder = '/content/GFPGAN/results_videos'  # .avi files
video_mp4_result_folder = '/content/GFPGAN/results_mp4_videos'  # .mp4 files
zee = 0

#deletes frames from previous video
#for f in os.listdir(upload_folder):
#    os.remove(os.path.join(upload_folder, f))

#deletes upscaled frames from previous video
#for f in os.listdir('results/restored_imgs'):
#    os.remove(os.path.join('results/restored_imgs', f))

# Clear previous .avi files
for f in os.listdir(video_result_folder):
    file_path = os.path.join(video_result_folder, f)
    if os.path.isfile(file_path):
        os.remove(file_path)
    elif os.path.isdir(file_path):
        shutil.rmtree(file_path)

# Clear previous .mp4 result files
for f in os.listdir(video_mp4_result_folder):
    file_path = os.path.join(video_mp4_result_folder, f)
    if os.path.isfile(file_path):
        os.remove(file_path)
    elif os.path.isdir(file_path):
        shutil.rmtree(file_path)


def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    #for sorting the file names properly
    files.sort(key = lambda x: int(x[5:-4]))
    size2 = (0,0)

    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        size2 = size
        print(filename)
        #inserting the frames into an image array
        frame_array.append(img)
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size2)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()


for filename in os.listdir(directory):

    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):


      print("PROCESSING :"+str(f)+"\n")
      # Read the video from specified path

      #video to frames
      cam = cv2.VideoCapture(str(f))

      # Get input resolution and FPS
      input_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
      input_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
      fps = cam.get(cv2.CAP_PROP_FPS)       #fps = 25.0 #change this to FPS of your source video
      print(f"Input resolution: {input_width}x{input_height}, FPS: {fps}")

      # Calculate scale factor for 4K (target 3840x2160)
      target_width, target_height = 3840, 2160
      scale_width = target_width / input_width
      scale_height = target_height / input_height
      scale = min(scale_width, scale_height)  # Preserve aspect ratio
      if scale < 1:
          scale = 1  # Avoid downscaling
      scale = round(scale)  # Use integer scale for simplicity
      print(f"Using scale factor: {scale}x")


      try:

          # PATH TO STORE VIDEO FRAMES
          if not os.path.exists('/content/GFPGAN/upload'):
              os.makedirs('/content/GFPGAN/upload')

      # if not created then raise error
      except OSError:
          print ('Error: Creating directory of data')

      # frame
      currentframe = 0

      #clear all folders


      #deletes upscaled frames from previous video
      #for f in os.listdir(result_folder):
      #  os.remove(os.path.join(result_folder, f))




      while(True):

          # reading from frame
          ret,frame = cam.read()

          if ret:
              # if video is still left continue creating images
              name = '/content/GFPGAN/upload/frame' + str(currentframe) + '.jpg'

              # writing the extracted images
              cv2.imwrite(name, frame)


                # increasing counter so that it will
                # show how many frames are created
              currentframe += 1
              print(currentframe)
          else:
              #deletes all the videos you uploaded for upscaling
              #for f in os.listdir(video_folder):
              #  os.remove(os.path.join(video_folder, f))

              break

        # Release all space and windows once done
      cam.release()
      #cv2.destroyAllWindows()

      frame_files = sorted(glob.glob(os.path.join(upload_folder, '*.jpg')))
      batch_size = 100
      for i in range(0, len(frame_files), batch_size):
          batch_dir = os.path.join(upload_folder, f'batch_{i}')
          os.makedirs(batch_dir, exist_ok=True)
          for frame in frame_files[i:i+batch_size]:
              shutil.move(frame, batch_dir)
          !python inference_gfpgan.py -i {batch_dir} -o results/batch_{i} -v 1.3 -s {scale} --bg_upsampler realesrgan --weight 0.6
          os.makedirs('results/restored_imgs', exist_ok=True)
          for processed_frame in glob.glob(f'results/batch_{i}/restored_imgs/*.jpg'):
              shutil.move(processed_frame, 'results/restored_imgs')
          shutil.rmtree(batch_dir)
          shutil.rmtree(f'results/batch_{i}')


      #after upscaling just delete the source frames
      for f in os.listdir(upload_folder):
        file_path = os.path.join(upload_folder, f)
        if os.path.isfile(file_path):
            os.remove(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)

      '''
      #rename all frames in "results" to remove the 'out' substring from the processing results
      paths = (os.path.join(root, filename)
              for root, _, filenames in os.walk('/content/GFPGAN/results')
              for filename in filenames)
      for path in paths:
          newname = path.replace('_out', '')
          if newname != path:
              os.rename(path, newname)
      '''

      #convert super res frames to .avi
      pathIn = '/content/GFPGAN/results/restored_imgs/'

      zee = zee+1
      fName = "video"+str(zee)
      base_filename = os.path.splitext(os.path.basename(filename))[0]
      filenameVid = f"{zee}_{base_filename}.avi" #filenameVid = f"{fName}.avi"
      pathOut = os.path.join(video_result_folder, filenameVid) #pathOut = "/content/GFPGAN/results_videos/"+filenameVid
      convert_frames_to_video(pathIn, pathOut, fps)

      #after processing frames converted to .avi video , delete upscaled frames from previous video
      for f in os.listdir('results/restored_imgs'):
          os.remove(os.path.join('results/restored_imgs', f))

      #convert .avi to .mp4
      src = video_result_folder
      dst = video_mp4_result_folder

      for root, dirs, filenames in os.walk(src, topdown=False):
          #print(filenames)
          for filename in filenames:
              print('[INFO] Processing ffmpeg',filename)
              try:
                  _format = ''
                  if ".flv" in filename.lower():
                      _format=".flv"
                  if ".mp4" in filename.lower():
                      _format=".mp4"
                  if ".avi" in filename.lower():
                      _format=".avi"
                  if ".mov" in filename.lower():
                        _format = ".mov"
                  inputfile = os.path.join(root, filename)
                  outputfile = os.path.join(dst, filename.lower().replace(_format, ".mp4"))
                  subprocess.call([
                        'ffmpeg', '-i', inputfile,
                        '-c:v', 'libx264', '-preset', 'slow', '-crf', '18',
                        '-an',  # no audio or '-c:a', 'aac', '-b:a', '192k',
                        '-vf', 'format=yuv420p',
                        outputfile
                  ])
              except:
                  print("An exception occurred")

      clear_output(wait=True)

      #clearing previous .avi files
      for f in os.listdir(video_result_folder):
          file_path = os.path.join(video_result_folder, f)
          if os.path.isfile(file_path):
              os.remove(file_path)
          elif os.path.isdir(file_path):
              shutil.rmtree(file_path)

      #deletes frames from previous video
      #for f in os.listdir(upload_folder):
      #  os.remove(os.path.join(upload_folder, f))

# Verify output resolution
for filename in os.listdir(video_mp4_result_folder):
    video_path = os.path.join(video_mp4_result_folder, filename)
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"Output video {filename}: {width}x{height}")
    cap.release()



      # if it is out of memory, try to use the `--tile` option
# We upsample the image with the scale factor X3.5

# Arguments
# -n, --model_name: Model names
# -i, --input: input folder or image
# --outscale: Output scale, can be arbitrary scale factore.

Output video 1_2fd3bcfe-a8dd-422a-9db8-f35c0cf1f461.mp4: 3840x2160


# 4. Download Results



*   All your upscaled .mp4 files will be inside the results_mp4_videos folder (inside GFPGAN folder)


*   You can right click on the needed file and download it from there.






# 5. Run this cell after the batch of videos have been upscaled
# (WARNING : Deletes processed data)



*   IMPORTANT : Read the comments inside the code section , because this DELETES previous frames and videos


In [ ]:


#deletes frames from previous video
for f in os.listdir(upload_folder):
    os.remove(os.path.join(upload_folder, f))

#deletes upscaled frames from previous video
for f in os.listdir('results/restored_imgs'):
    os.remove(os.path.join('results/restored_imgs', f))

#deletes all the videos you uploaded for upscaling
for f in os.listdir(upload_folder):
    file_path = os.path.join(upload_folder, f)
    if os.path.isfile(file_path):
        os.remove(file_path)
    elif os.path.isdir(file_path):
        shutil.rmtree(file_path)

#clearing previous .avi files
for f in os.listdir(video_result_folder):
    os.remove(os.path.join(video_result_folder, f))

#clearing .mp4 result files
for f in os.listdir(video_mp4_result_folder):
    os.remove(os.path.join(video_mp4_result_folder, f))